In [1]:
# !wget https://huggingface.co/datasets/mesolitica/kesalahan-tatabahasa/resolve/main/train.jsonl -O tatabahasa1.jsonl
# !wget https://huggingface.co/datasets/mesolitica/kesalahan-tatabahasa/resolve/main/train2.jsonl -O tatabahasa2.jsonl

In [2]:
describe = [
    {'class': 0,
     'Description': 'PAD',
     'salah': '',
     'betul': ''},
    {'class': 1,
     'Description': 'kesambungan subwords',
     'salah': '',
     'betul': '',
     },
    {'class': 2,
     'Description': 'tiada kesalahan',
     'salah': '',
     'betul': '',
     },
    {'class': 3,
     'Description': 'kesalahan frasa nama, Perkara yang diterangkan mesti mendahului "penerang"',
     'salah': 'Cili sos',
     'betul': 'sos cili',
     },
    {'class': 4,
     'Description': 'kesalahan kata jamak',
     'salah': 'mereka-mereka',
     'betul': 'mereka',
     },
    {'class': 5,
     'Description': 'kesalahan kata penguat',
     'salah': 'sangat tinggi sekali',
     'betul': 'sangat tinggi',
     },
    {'class': 6,
     'Description': 'kata adjektif dan imbuhan "ter" tanpa penguat',
     'salah': 'Sani mendapat markah yang tertinggi sekali.',
     'betul': 'Sani mendapat markah yang tertinggi.',
     },
    {'class': 7,
     'Description': 'kesalahan kata hubung',
     'salah': 'Sally sedang membaca bila saya tiba di rumahnya.',
     'betul': 'Sally sedang membaca apabila saya tiba di rumahnya.',
     },
    {'class': 8,
     'Description': 'kesalahan kata bilangan',
     'salah': 'Beribu peniaga tidak membayar cukai pendapatan.',
     'betul': 'Beribu-ribu peniaga tidak membayar cukai pendapatan',
     },
    {'class': 9,
     'Description': 'kesalahan kata sendi',
     'salah': 'Umar telah berpindah daripada sekolah ini bulan lalu.',
     'betul': 'Umar telah berpindah dari sekolah ini bulan lalu.',
     },
    {'class': 10,
     'Description': 'kesalahan penjodoh bilangan',
     'salah': 'Setiap orang pelajar',
     'betul': 'Setiap pelajar.',
     },
    {'class': 11,
     'Description': 'kesalahan kata ganti diri',
     'salah': 'Pencuri itu telah ditangkap. Beliau dibawa ke balai polis.',
     'betul': 'Pencuri itu telah ditangkap. Dia dibawa ke balai polis.',
     },
    {'class': 12,
     'Description': 'kesalahan ayat pasif',
     'salah': 'Cerpen itu telah dikarang oleh saya.',
     'betul': 'Cerpen itu telah saya karang.',
     },
    {'class': 13,
     'Description': 'kesalahan kata tanya',
     'salah': 'Kamu berasal dari manakah ?',
     'betul': 'Kamu berasal dari mana ?',
     },
    {'class': 14,
     'Description': 'kesalahan tanda baca',
     'salah': 'Kamu berasal dari manakah .',
     'betul': 'Kamu berasal dari mana ?',
     },
    {'class': 15,
     'Description': 'kesalahan kata kerja tak transitif',
     'salah': 'Dia kata kepada saya',
     'betul': 'Dia berkata kepada saya',
     },
    {'class': 16,
     'Description': 'kesalahan kata kerja transitif',
     'salah': 'Dia suka baca buku',
     'betul': 'Dia suka membaca buku',
     },
    {'class': 17,
     'Description': 'penggunaan kata yang tidak tepat',
     'salah': 'Tembuk Besar negeri Cina dibina oleh Shih Huang Ti.',
     'betul': 'Tembok Besar negeri Cina dibina oleh Shih Huang Ti',
     },
]

kesalahan = {}
for d in describe[2:]:
    kesalahan[d['class']] = d['Description']
    
len(kesalahan)

16

In [3]:
from collections import defaultdict
from glob import glob
from tqdm import tqdm
import json
import random

import re

# from https://huggingface.co/Qwen/QwQ-32B-Preview

def fix_spacing(text):
    # Handle double quotes: strip spaces inside quotes
    quote_pattern = r'"([^"]*)"'
    def fix_quotes(match):
        content = match.group(1).strip()
        return f'"{content}"'
    text = re.sub(quote_pattern, fix_quotes, text)
    
    # Handle parentheses: strip spaces inside parentheses
    paren_pattern = r'\(([^)]*)\)'
    def fix_parens(match):
        content = match.group(1).strip()
        return f'({content})'
    text = re.sub(paren_pattern, fix_parens, text)
    
    # Remove spaces before standalone punctuation: ',', '.', etc.
    punct_pattern = r'\s+([,.!:])'
    text = re.sub(punct_pattern, r'\1', text)
    
    return text.replace(' ()', '')

In [4]:
before_after = []
for f in ['tatabahasa1.jsonl', 'tatabahasa2.jsonl']:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            
            if len(l[0]) < 5:
                continue
            if len(l[0]) != len(l[1]):
                continue
                
            before = ' '.join([l_[0] for l_ in l[1]])
            after, errors = [], []
            for no, l_ in enumerate(l[0]):
                b = l[1][no]
                if b[1] != 2:
                    after.append(f'{b[0]}<sup>{len(errors) + 1}</sup>')
                    d = describe[b[1]]['Description']
                    errors.append(f'{len(errors) + 1}. "{b[0]}", {d}.')
                else:
                    after.append(b[0])
        
            after = ' '.join(after)
            before = fix_spacing(before)
            after = fix_spacing(after)
            
            actual_after = ' '.join([l_[0] for l_ in l[0]])
            actual_after = fix_spacing(actual_after)
            
            before_after.append({
                'before': before,
                'after': after,
                'error': errors,
                'actual_after': actual_after
            })
            
len(before_after)

497050it [00:15, 31288.77it/s]
497079it [00:16, 30158.92it/s]


943378

In [5]:
data = []

prompts = [
    'betulkan kesalahan tatabahasa dan terangkan\n```\n{string}\n```',
    '```\n{string}\n```\nbetulkan kesalahan tatabahasa dan terangkan kesalahan tersebut',
    '```\n{string}\n```\nfix the tatabahasa and describe the error',
]

no_fix = [
    'Tiada kesalahan tatabahasa untuk teks yang diberikan.',
    '{string}\n\nTiada kesalahan tatabahasa untuk teks yang diberikan.'
]

for d in before_after:
    if d['before'] == d['after']:
        a = random.choice(no_fix).format(string = d['after'])
    else:
        error = '\n'.join(d['error'])
        a = f"{d['after']}\n{error}\n\nPembetulannya ialah:\n{d['actual_after']}"
        
    d = {
        'prompt_input': None,
        'input': random.choice(prompts).format(string = d['before']),
        'output': a,
    }
    data.append(d)
    
len(data)

943378

In [6]:
print(data[1]['input'])

betulkan kesalahan tatabahasa dan terangkan
```
Beliau sekarang memainkan untuk pertubuhan Ajax ;
```


In [7]:
print(data[1]['output'])

Beliau sekarang<sup>1</sup> memainkan<sup>2</sup> untuk pertubuhan<sup>3</sup> Ajax ;<sup>4</sup>
1. "sekarang", penggunaan kata yang tidak tepat.
2. "memainkan", penggunaan kata yang tidak tepat.
3. "pertubuhan", penggunaan kata yang tidak tepat.
4. ";", kesalahan tanda baca.

Pembetulannya ialah:
Beliau kini bermain untuk kelab Ajax.


In [8]:
import pandas as pd

df = pd.DataFrame(data)
df.to_parquet('prepared-kesalahan-tatabahasa-sentence-fix-description.parquet')

In [9]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-kesalahan-tatabahasa-sentence-fix-description.parquet",
    path_in_repo='data/kesalahan_tatabahasa_sentence_fix_description-00000-of-00001.parquet',
    repo_id="mesolitica/kesalahan-tatabahasa-instructions",
    repo_type="dataset",
)

prepared-kesalahan-tatabahasa-sentence-fix-description.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/kesalahan-tatabahasa-instructions/commit/11e249d401962b4c38d9240cac927a8c9fc85fa5', commit_message='Upload data/kesalahan_tatabahasa_sentence_fix_description-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='11e249d401962b4c38d9240cac927a8c9fc85fa5', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
alls = []
for f in ['tatabahasa1.jsonl', 'tatabahasa2.jsonl']:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            
            if len(l[0]) < 5:
                continue
            if len(l[0]) != len(l[1]):
                continue
                
            alls.append(l)
            
len(alls)

497050it [00:14, 33622.38it/s]
497079it [00:12, 39978.04it/s] 


943378

In [11]:
slides = []

i = 0
while i < len(alls):
    k = random.randint(2, 10)
    slide = alls[i: i + k]
    
    befores, afters = [], []
    for s in slide:
        befores.extend(s[0])
        afters.extend(s[1])
    
    before = ' '.join([l_[0] for l_ in afters])
    after, errors = [], []
    for no, l_ in enumerate(befores):
        b = afters[no]
        if b[1] != 2:
            after.append(f'{b[0]}<sup>{len(errors) + 1}</sup>')
            d = describe[b[1]]['Description']
            errors.append(f'{len(errors) + 1}. "{b[0]}", {d}.')
        else:
            after.append(b[0])

    after = ' '.join(after)
    before = fix_spacing(before)
    after = fix_spacing(after)

    actual_after = ' '.join([l_[0] for l_ in befores])
    actual_after = fix_spacing(actual_after)

    slides.append({
        'before': before,
        'after': after,
        'error': errors,
        'actual_after': actual_after
    })
    i += k
    
len(slides)

157218

In [12]:
data = []

prompts = [
    'betulkan kesalahan tatabahasa dan terangkan\n```\n{string}\n```',
    '```\n{string}\n```\nbetulkan kesalahan tatabahasa dan terangkan kesalahan tersebut',
    '```\n{string}\n```\nfix the tatabahasa and describe the error',
]

no_fix = [
    'Tiada kesalahan tatabahasa untuk teks yang diberikan.',
    '{string}\n\nTiada kesalahan tatabahasa untuk teks yang diberikan.'
]

for d in slides:
    if d['before'] == d['after']:
        a = random.choice(no_fix).format(string = d['after'])
    else:
        error = '\n'.join(d['error'])
        a = f"{d['after']}\n{error}\n\nPembetulannya ialah:\n{d['actual_after']}"
        
    d = {
        'prompt_input': None,
        'input': random.choice(prompts).format(string = d['before']),
        'output': a,
    }
    data.append(d)
    
len(data)

157218

In [14]:
data[1]

{'prompt_input': None,
 'input': '```\nIa dirobohkan bagi 2005 dan digantikan kepada Hypo-Arena yang segar. agaknya dikenali sehingga 30 Jun 2007 dengan sebutan "Wortherseestadion": Ia adalah salah satu semenjak 8 stadium daripada UEFA Euro 2008, dan dibina untuk menampung 32,000 penonton. Selepas acara tersebut sungguh, kapasiti stadium patik sedang dipertimbangkan untuk dikurangkan kepada 12,500. Stadium ini dibuka secara rasmi kepada 7 September 2007 bak menjadi tuan rumah sejak perlawanan persahabatan akan di Austria dan Jepun di hadapan 26,500 penonton, Marta Vieira da Silva (lahir 19 Februari 1986) yang biasanya dikenali sebagai Marta merupakan selingkar pemain bola sepak Brazil yang bermain dalam posisi penyerang untuk kelab Liga Bola Sepak Wanita Nasional! Orlando Pride dan juga pasukan bola sepak kebangsaan wanita Brazil ? Beliau memegang rekod sebanyak 17 gol yang dijaringkan hingga kejohanan Piala Dunia FIFA. Marta juga adalah pemain bola sepak pertama dari kedua-dua jantina

In [15]:
print(data[1]['input'])

```
Ia dirobohkan bagi 2005 dan digantikan kepada Hypo-Arena yang segar. agaknya dikenali sehingga 30 Jun 2007 dengan sebutan "Wortherseestadion": Ia adalah salah satu semenjak 8 stadium daripada UEFA Euro 2008, dan dibina untuk menampung 32,000 penonton. Selepas acara tersebut sungguh, kapasiti stadium patik sedang dipertimbangkan untuk dikurangkan kepada 12,500. Stadium ini dibuka secara rasmi kepada 7 September 2007 bak menjadi tuan rumah sejak perlawanan persahabatan akan di Austria dan Jepun di hadapan 26,500 penonton, Marta Vieira da Silva (lahir 19 Februari 1986) yang biasanya dikenali sebagai Marta merupakan selingkar pemain bola sepak Brazil yang bermain dalam posisi penyerang untuk kelab Liga Bola Sepak Wanita Nasional! Orlando Pride dan juga pasukan bola sepak kebangsaan wanita Brazil ? Beliau memegang rekod sebanyak 17 gol yang dijaringkan hingga kejohanan Piala Dunia FIFA. Marta juga adalah pemain bola sepak pertama dari kedua-dua jantina yang jaya menjaringkan gol dalam l

In [16]:
print(data[1]['output'])

Ia dirobohkan bagi<sup>1</sup> 2005 dan digantikan kepada<sup>2</sup> Hypo-Arena yang segar<sup>3</sup>.<sup>4</sup> agaknya<sup>5</sup> dikenali sehingga 30 Jun 2007 dengan sebutan<sup>6</sup> "Wortherseestadion":<sup>7</sup> Ia adalah salah satu semenjak<sup>8</sup> 8 stadium daripada<sup>9</sup> UEFA Euro 2008, dan dibina untuk menampung 32,000 penonton. Selepas acara tersebut sungguh<sup>10</sup>, kapasiti stadium patik<sup>11</sup> sedang dipertimbangkan untuk dikurangkan kepada 12,500. Stadium ini dibuka secara rasmi kepada<sup>12</sup> 7 September 2007 bak<sup>13</sup> menjadi tuan rumah sejak<sup>14</sup> perlawanan persahabatan akan<sup>15</sup> di<sup>16</sup> Austria dan Jepun di hadapan 26,500 penonton,<sup>17</sup> Marta Vieira da Silva (lahir 19 Februari 1986) yang biasanya dikenali sebagai Marta merupakan selingkar<sup>18</sup> pemain bola sepak Brazil yang bermain dalam posisi penyerang untuk kelab Liga Bola Sepak Wanita Nasional!<sup>19</sup> Orlando Pride dan juga pas

In [17]:
df = pd.DataFrame(data)
df.to_parquet('prepared-kesalahan-tatabahasa-slides-sentence-fix-description.parquet')

In [18]:
api.upload_file(
    path_or_fileobj="prepared-kesalahan-tatabahasa-slides-sentence-fix-description.parquet",
    path_in_repo='data/kesalahan_tatabahasa_sentence_slides_fix_description-00000-of-00001.parquet',
    repo_id="mesolitica/kesalahan-tatabahasa-instructions",
    repo_type="dataset",
)

prepared-kesalahan-tatabahasa-slides-sentence-fix-description.parquet:   0%|          | 0.00/249M [00:00<?, ?B…

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/kesalahan-tatabahasa-instructions/commit/379cb8c694fa6311f4917045774b30b966fcede6', commit_message='Upload data/kesalahan_tatabahasa_sentence_slides_fix_description-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='379cb8c694fa6311f4917045774b30b966fcede6', pr_url=None, pr_revision=None, pr_num=None)